## Data Cleaning for MTA DB

Notebook to clean project database.

In [543]:
pd.set_option('display.float_format', lambda x: '%.f' % x)

In [461]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import mtafunctions

In [556]:
from mtafunctions import *

In [555]:
from importlib import reload
reload(mtafunctions)

<module 'mtafunctions' from '/Users/joycetagal/Github/metis-eda/mtafunctions.py'>

In [464]:
engine = create_engine("sqlite:///mta_data.db")

In [465]:
engine.table_names()

['mta_data']

## Data cleaning
### General data cleaning

In [466]:
turnstiles_df = pd.read_sql('SELECT * FROM mta_data', engine)

In [467]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/14/2021,00:00:00,REGULAR,7618705,2606187
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/14/2021,04:00:00,REGULAR,7618717,2606191
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/14/2021,08:00:00,REGULAR,7618732,2606216
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/14/2021,12:00:00,REGULAR,7618788,2606260
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/14/2021,16:00:00,REGULAR,7618905,2606291


In [468]:
# Parse datetime column
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

In [469]:
# Get rid of duplicate entries
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [504]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
209414,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,21:00:00,REGULAR,5554,613,2021-08-20 21:00:00
209413,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,17:00:00,REGULAR,5554,613,2021-08-20 17:00:00
209412,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,13:00:00,REGULAR,5554,613,2021-08-20 13:00:00
209411,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,09:00:00,REGULAR,5554,613,2021-08-20 09:00:00
209410,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,05:00:00,REGULAR,5554,613,2021-08-20 05:00:00


In [505]:
# Remove any dates before 2020
turnstiles_df = turnstiles_df[turnstiles_df['DATE_TIME'] > '01/01/2020']

In [506]:
# Sanity check for duplicates
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2020-12-26 03:00:00,1
4744046,R138,R293,00-03-04,34 ST-PENN STA,2021-04-30 10:00:00,1
4744057,R138,R293,00-03-04,34 ST-PENN STA,2021-05-02 06:00:00,1
4744056,R138,R293,00-03-04,34 ST-PENN STA,2021-05-02 02:00:00,1
4744055,R138,R293,00-03-04,34 ST-PENN STA,2021-05-01 22:00:00,1


In [508]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
209414,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,21:00:00,REGULAR,5554,613,2021-08-20 21:00:00
209413,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,17:00:00,REGULAR,5554,613,2021-08-20 17:00:00
209412,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,13:00:00,REGULAR,5554,613,2021-08-20 13:00:00
209411,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,09:00:00,REGULAR,5554,613,2021-08-20 09:00:00
209410,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/20/2021,05:00:00,REGULAR,5554,613,2021-08-20 05:00:00


In [510]:
turnstiles_df.shape[0]

7116075

### Daily exits and entries per turnstile

In [511]:
station_strkeep = ['1 AV', '34 ST-PENN STA', '34 ST-HERALD SQ', '33 ST', '28 ST', '23 ST', '18 ST', '14 ST', 
                   '14 ST-UNION SQ', '8 ST-NYU', 'ASTOR PL', '3 AV', 'W 4 ST-WASH SQ', 'BLEECKER ST', "B'WAY-LAFAYETTE",
                   '2 AV', '34 ST-HUDSON YD', 'DELANCEY/ESSEX', 'PRINCE ST', 'CHRISTOPHER ST', 'HOUSTON ST', 'SPRING ST',
                   'CANAL ST', 'BOWERY', 'GRAND ST', 'EAST BROADWAY', 'CITY HALL', 'CORTLANDT ST', 'RECTOR ST', 'FRANKLIN ST',
                   'CHAMBERS ST', 'BROOKLYN BRIDGE', 'FULTON ST', 'WALL ST', 'PARK PLACE', 'WORLD TRADE CTR', 'BOWLING GREEN',
                   'SOUTH FERRY', 'WHITEHALL S-FRY'                  
                  ]

In [564]:
turnstiles_daily = turnstiles_df[turnstiles_df['STATION'].isin(station_strkeep)]

In [565]:
turnstiles_daily.STATION.unique()

array(['34 ST-HUDSON YD', '33 ST', '28 ST', '23 ST', '14 ST-UNION SQ',
       'ASTOR PL', 'BLEECKER ST', 'SPRING ST', 'BROOKLYN BRIDGE',
       'FULTON ST', 'WALL ST', 'BOWLING GREEN', '34 ST-PENN STA', '18 ST',
       '14 ST', 'CHRISTOPHER ST', 'HOUSTON ST', 'CANAL ST', 'FRANKLIN ST',
       'CHAMBERS ST', 'PARK PLACE', 'RECTOR ST', 'SOUTH FERRY',
       'EAST BROADWAY', 'DELANCEY/ESSEX', '2 AV', 'GRAND ST',
       "B'WAY-LAFAYETTE", '34 ST-HERALD SQ', 'WORLD TRADE CTR',
       'W 4 ST-WASH SQ', '1 AV', '3 AV', 'BOWERY', 'WHITEHALL S-FRY',
       'CORTLANDT ST', 'CITY HALL', 'PRINCE ST', '8 ST-NYU'], dtype=object)

In [566]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
197037,R551,R072,01-05-01,34 ST-HUDSON YD,7,IRT,08/20/2021,20:00:00,REGULAR,1,150,2021-08-20 20:00:00
197036,R551,R072,01-05-01,34 ST-HUDSON YD,7,IRT,08/20/2021,16:00:00,REGULAR,1,150,2021-08-20 16:00:00
197035,R551,R072,01-05-01,34 ST-HUDSON YD,7,IRT,08/20/2021,12:00:00,REGULAR,1,150,2021-08-20 12:00:00
197034,R551,R072,01-05-01,34 ST-HUDSON YD,7,IRT,08/20/2021,08:00:00,REGULAR,1,150,2021-08-20 08:00:00
197033,R551,R072,01-05-01,34 ST-HUDSON YD,7,IRT,08/20/2021,04:00:00,REGULAR,1,150,2021-08-20 04:00:00


In [567]:
turnstiles_daily = (turnstiles_daily
                    .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"], as_index=False)
                    [['ENTRIES', 'EXITS']].first()
                   )

In [568]:
turnstiles_daily.shape[0]

255470

In [569]:
turnstiles_daily['DATE'] = pd.to_datetime(turnstiles_daily.DATE, format="%m/%d/%Y")

In [570]:
turnstiles_daily.dtypes

C/A                object
UNIT               object
SCP                object
STATION            object
DATE       datetime64[ns]
ENTRIES             int64
EXITS               int64
dtype: object

In [571]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,EXITS
0,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-01,28688647,12101699
1,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-02,28689361,12102479
2,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-03,28689771,12102932
3,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-04,28690839,12103784
4,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-05,28691968,12104640


In [572]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES", "PREV_EXITS"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])[["DATE", "ENTRIES", "EXITS"]]
                                                       .apply(lambda grp: grp.shift(1)))

In [573]:
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [574]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS
1,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-02,28689361,12102479,2021-01-01,28688647,12101699
2,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-03,28689771,12102932,2021-01-02,28689361,12102479
3,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-04,28690839,12103784,2021-01-03,28689771,12102932
4,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-05,28691968,12104640,2021-01-04,28690839,12103784
5,A022,R022,01-00-00,34 ST-HERALD SQ,2021-01-06,28693036,12105418,2021-01-05,28691968,12104640


In [583]:
turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_entries, axis=1, max_counter=500000)

In [584]:
turnstiles_daily.sort_values('DAILY_ENTRIES', ascending=False).head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,DAILY_ENTRIES,DAILY_EXITS
149772,PTH09,R548,00-00-02,CHRISTOPHER ST,2020-12-26,527950,642892,2021-08-20,59394,80469,468556,80469
89440,N083,R138,01-05-00,W 4 ST-WASH SQ,2021-03-31,0,0,2021-03-22,458975,0,458975,0
89580,N083,R138,01-05-00,W 4 ST-WASH SQ,2020-12-26,458962,0,2021-08-20,87,0,458875,0
116439,N422,R318,00-05-00,FULTON ST,2021-07-29,327682,9,2021-07-27,786526,0,458844,9
116462,N422,R318,00-05-00,FULTON ST,2020-12-26,786502,0,2021-08-20,327686,9,458816,9


In [585]:
turnstiles_daily["DAILY_EXITS"] = turnstiles_daily.apply(get_daily_exits, axis=1, max_counter=500000)

In [586]:
turnstiles_daily.sort_values('DAILY_EXITS', ascending=False).head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,DAILY_ENTRIES,DAILY_EXITS
3326,A025,R023,01-06-01,34 ST-HERALD SQ,2020-12-26,15657523,32914210,2021-08-20,15831308,33412216,173785,498006
87929,N083,R138,01-00-01,W 4 ST-WASH SQ,2020-12-26,293189,414875,2021-08-20,527074,910707,233885,495832
120267,N506,R022,00-00-00,34 ST-HERALD SQ,2020-12-26,8176247,14400612,2021-08-20,8414208,14894244,237961,493632
11180,A034,R170,03-00-00,14 ST-UNION SQ,2020-12-26,2265749,17728682,2021-08-20,2321142,18219985,55393,491303
69789,N068,R012,03-00-02,34 ST-PENN STA,2021-07-14,5946,3870,2021-07-13,4298704,494138,5946,490268


In [587]:
station_daily = turnstiles_daily.groupby(["STATION", "DATE"])[['DAILY_ENTRIES', 'DAILY_EXITS']].sum().reset_index()

In [588]:
station_daily.sort_values('DAILY_ENTRIES', ascending=False).head()

,STATION,DATE,DAILY_ENTRIES,DAILY_EXITS
2607,34 ST-PENN STA,2020-12-26,9238459,10161605
2133,34 ST-HERALD SQ,2020-12-26,6733889,7127782
6399,FULTON ST,2020-12-26,6032918,6047735
1185,23 ST,2020-12-26,5573504,6880417
474,14 ST-UNION SQ,2020-12-26,5071993,7169072


In [589]:
station_daily.shape

(9243, 4)

In [621]:
stationpoints = pd.read_csv('/Users/joycetagal/Github/metis-eda/DOITT_SUBWAY_STATION_01_13SEPT2010.csv')
stationpoints['NAME'] = stationpoints['NAME'].str.upper()

In [622]:
stationpoints.size

2838

In [623]:
## Take the first location for each station (to simplify)
stationpoints = stationpoints.groupby('NAME', as_index=False).first()

In [625]:
stationpoints.head()

,NAME,URL,OBJECTID,the_geom,LINE,NOTES
0,1 AV,http://web.mta.info/nyct/service/,146,POINT (-73.98168087489128 40.73097497580066),L,L-all times
1,103RD ST,http://web.mta.info/nyct/service/,159,POINT (-73.96837899960818 40.799446000334825),1,1-all times
2,103RD ST - CORONA PLAZA,http://web.mta.info/nyct/service/,189,POINT (-73.86269999830412 40.749865000555545),7,7-all times
3,104TH ST,http://web.mta.info/nyct/service/,98,POINT (-73.83768300060997 40.681711001091195),A-S,S Euclid Av to Ozone Park-Lefferts Blvd-nights...
4,104TH-102ND STS,http://web.mta.info/nyct/service/,14,POINT (-73.84443500029684 40.69516599823373),J-Z,"Z-rush hours AM westbound, PM eastbound, J-all..."


In [624]:
stationpoints[stationpoints['NAME'] == "1 AV"]

,NAME,URL,OBJECTID,the_geom,LINE,NOTES
0,1 AV,http://web.mta.info/nyct/service/,146,POINT (-73.98168087489128 40.73097497580066),L,L-all times


In [626]:
station_csv_names = stationpoints[stationpoints['NAME'].isin(station_strkeep)]['NAME']
station_csv_names

0                 1 AV
14               14 ST
15      14 ST-UNION SQ
32               18 ST
39                2 AV
47               23 ST
54               28 ST
56                3 AV
58               33 ST
60     34 ST-HERALD SQ
61     34 ST-HUDSON YD
62      34 ST-PENN STA
104           8 ST-NYU
119           ASTOR PL
132    B'WAY-LAFAYETTE
152        BLEECKER ST
155             BOWERY
156      BOWLING GREEN
165    BROOKLYN BRIDGE
171           CANAL ST
179        CHAMBERS ST
181     CHRISTOPHER ST
183          CITY HALL
190       CORTLANDT ST
200     DELANCEY/ESSEX
207      EAST BROADWAY
221        FRANKLIN ST
225          FULTON ST
231           GRAND ST
241         HOUSTON ST
304         PARK PLACE
312          PRINCE ST
318          RECTOR ST
329        SOUTH FERRY
330          SPRING ST
346     W 4 ST-WASH SQ
350            WALL ST
353    WHITEHALL S-FRY
361    WORLD TRADE CTR
Name: NAME, dtype: object

In [627]:
set_difference = set(station_strkeep) - set(station_csv_names)
list_difference = list(set_difference)
list_difference

[]

In [629]:
stations_lower = station_daily[station_daily['STATION'].isin(station_strkeep)]

In [630]:
merged_station_data = stations_lower.merge(stationpoints, left_on='STATION', right_on='NAME')

In [631]:
merged_station_data.sort_values('NAME', ascending=True).head()

,STATION,DATE,DAILY_ENTRIES,DAILY_EXITS,NAME,URL,OBJECTID,the_geom,LINE,NOTES
0,1 AV,2020-12-26,1910414,2649351,1 AV,http://web.mta.info/nyct/service/,146,POINT (-73.98168087489128 40.73097497580066),L,L-all times
151,1 AV,2021-05-27,7931,13262,1 AV,http://web.mta.info/nyct/service/,146,POINT (-73.98168087489128 40.73097497580066),L,L-all times
152,1 AV,2021-05-28,8252,13797,1 AV,http://web.mta.info/nyct/service/,146,POINT (-73.98168087489128 40.73097497580066),L,L-all times
153,1 AV,2021-05-29,5469,9930,1 AV,http://web.mta.info/nyct/service/,146,POINT (-73.98168087489128 40.73097497580066),L,L-all times
154,1 AV,2021-05-30,4575,8830,1 AV,http://web.mta.info/nyct/service/,146,POINT (-73.98168087489128 40.73097497580066),L,L-all times


In [633]:
len(merged_station_data['NAME'].unique())

39

In [634]:
merged_station_data.to_csv('station_daily_locs.csv')